## 1. Interploate the missing dates.

The following dates are missing in the dataset:



In [1]:
import pandas as pd

# Load the CSV file
basin_id = "05535070"
file_path = f'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/US_climate_discharge_data/{basin_id}.csv'
df = pd.read_csv(file_path)
df.rename(columns={"Date": "date"}, inplace=True)

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Set the 'date' column as the index
df.set_index('date', inplace=True)

# Generate a date range from the minimum to the maximum date in the dataframe
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

# Reindex the dataframe to this date range
df_reindexed = df.reindex(date_range)

# Find missing dates
missing_dates = df_reindexed[df_reindexed.isnull().any(axis=1)].index

# Print missing dates
print("Missing dates:")
for date in missing_dates:
    print(date.strftime('%Y-%m-%d'))

# Check frequency
frequency = pd.infer_freq(df.index)
print(f"Detected frequency: {frequency}")

Missing dates:
1980-12-31
1984-12-31
1988-12-31
1992-12-31
1996-12-31
2000-12-31
2004-12-31
2008-12-31
2012-12-31
2016-12-31
2020-12-31
Detected frequency: None


Therefore, we will interpolate the missing dates in the dataset.

In [17]:
import os
import pandas as pd
from tqdm import tqdm

# Define the input and output folders
input_folder = 'climate_discharge_data'
output_folder = 'data/csv_files'
basins_list_file = 'basin_list/basin_list_complete.txt'

# Read the list of basin IDs
with open(basins_list_file, 'r') as file:
    basin_ids = file.read().splitlines()

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to interpolate missing dates
def interpolate_missing_dates(df, missing_dates):
    # Ensure the 'date' column is in datetime format
    df.rename(columns={"Date": "date"}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    # Set the 'date' column as the index
    df.set_index('date', inplace=True)
    # Generate a complete date range from the minimum to the maximum date in the dataframe
    date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    # Reindex the dataframe to this date range
    df_reindexed = df.reindex(date_range)
    # Interpolate missing values
    df_interpolated = df_reindexed.interpolate(method='time')
    return df_interpolated

# Process each CSV file
for basin_id in tqdm(basin_ids):
    input_file = os.path.join(input_folder, f'{basin_id}.csv')
    output_file = os.path.join(output_folder, f'{basin_id}.csv')
    
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Interpolate missing dates
    missing_dates = [
        '1980-12-31', '1984-12-31', '1988-12-31', '1992-12-31', 
        '1996-12-31', '2000-12-31', '2004-12-31', '2008-12-31', 
        '2012-12-31', '2016-12-31', '2020-12-31'
    ]
    df_interpolated = interpolate_missing_dates(df, missing_dates)
    
    # Reset the index to have 'date' as a column again
    df_interpolated.reset_index(inplace=True)
    df_interpolated.rename(columns={'index': 'date'}, inplace=True)
    
    # Convert all negative values in the "discharge" column to NaN
    df_interpolated['discharge'] = df_interpolated['discharge'].apply(lambda x: x if x >= 0 else pd.NA)
    
    # If the basin_id does not start with "02", divide the values in the "discharge" column by 35.315
    # Basins not starting with "02" are US stations, using an undesired unit - cubic ft / s
    if not basin_id.startswith("02"):
        df_interpolated['discharge'] = df_interpolated['discharge'] / 35.315
    
    # Write the output CSV file
    df_interpolated.to_csv(output_file, index=False)

print("Interpolation and file writing completed.")

100%|██████████| 976/976 [01:19<00:00, 12.20it/s]

Interpolation and file writing completed.


## 2. Check the data for missing dates.

In [18]:
import os
import pandas as pd

# Define the directory containing the imputed CSV files
directory = 'data/csv_files'

# Initialize a flag to check if all files meet the condition
all_files_valid = True
non_satisfying_files = []

# Get the list of all file names in the directory
basin_ids = [f.split('.')[0] for f in os.listdir(directory) if f.endswith('.csv')]

# Iterate over each file in the directory
for basin_id in tqdm(basin_ids):
    
    file_path = f'data/csv_files/{basin_id}.csv'
    file_name = f'{basin_id}.csv'
        
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    df.rename(columns={"Date": "date"}, inplace=True)
    # Ensure the 'date' column is in datetime format and set as index
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.set_index('date', inplace=True)
    
    # Create a full date range and check for missing dates
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    missing_dates = full_range.difference(df.index)
    
    # Check if the frequency of missing dates is 'D'
    if missing_dates.freq != 'D':
        # print(f"File {file_name} does not meet the condition.")
        # print(f"Frequency of missing dates: {missing_dates.freq}")
        all_files_valid = False
        non_satisfying_files.append(file_name)
        

if all_files_valid:
    print("All files meet the condition.")
else:
    print("Some files do not meet the condition.")

100%|██████████| 976/976 [00:16<00:00, 60.14it/s]

All files meet the condition.


## 3. Convert the data to a time series. (NetCDF)

In [19]:
from file_conversion import DataFrame_to_CDF
from tqdm import tqdm
import pandas as pd
import os

def convert_csv_to_cdf(input_folder: str, output_folder: str):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all CSV files in the input folder
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    
    for csv_file in tqdm(csv_files):
        # Construct the file path for the CSV file
        csv_file_path = os.path.join(input_folder, csv_file)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path, index_col='date')
        df.index = pd.to_datetime(df.index)
        
        # Convert the DataFrame to a CDF file
        basin_id = os.path.splitext(csv_file)[0]
        DataFrame_to_CDF(df, output_folder, basin_id)


In [20]:
input_folder = 'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/data/csv_files'
output_folder = 'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/data/time_series'
convert_csv_to_cdf(input_folder, output_folder)

100%|██████████| 976/976 [00:28<00:00, 34.36it/s]


## 4. Model Training

In [21]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

In [27]:
start_run(config_file=Path('configs/config.yml'))

2025-02-11 11:04:23,100: Logging to c:\Users\ybrot\Desktop\course\UROP\GAGEii_modeling\runs\GAGEii-TransformerTest_1102_110423\output.log initialized.
2025-02-11 11:04:23,100: ### Folder structure created at c:\Users\ybrot\Desktop\course\UROP\GAGEii_modeling\runs\GAGEii-TransformerTest_1102_110423
2025-02-11 11:04:23,100: ### Run configurations for GAGEii-TransformerTest
2025-02-11 11:04:23,101: experiment_name: GAGEii-TransformerTest
2025-02-11 11:04:23,101: run_dir: c:\Users\ybrot\Desktop\course\UROP\GAGEii_modeling\runs\GAGEii-TransformerTest_1102_110423
2025-02-11 11:04:23,102: train_basin_file: basin_list\missing_involved.txt
2025-02-11 11:04:23,102: validation_basin_file: basin_list\missing_involved.txt
2025-02-11 11:04:23,103: test_basin_file: basin_list\missing_involved.txt
2025-02-11 11:04:23,103: train_start_date: 1981-01-01 00:00:00
2025-02-11 11:04:23,103: train_end_date: 2000-12-31 00:00:00
2025-02-11 11:04:23,104: validation_start_date: 2001-01-01 00:00:00
2025-02-11 11:0

ValueError: transformer_nheads is not specified in the config (.yml).

# Evaluation

In [12]:
run_dir = Path("runs/GAGEii-MissingValueTest_0502_082337") 
eval_run(run_dir=run_dir, period="test")

with open(run_dir / "test" / "model_epoch050" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)

2025-02-05 09:06:07,041: Using the model weights from runs\GAGEii-MissingValueTest_0502_082337\model_epoch050.pt


C:\Users\ybrot\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\neuralhydrology\evaluation\tester.py:133: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f

# Evaluation: 100%|██████████| 1/1 [00:00<00:00,  2.09it/s]
2025-02-05 09:06:07,526: Stored metrics at runs\GAGEii-MissingValueTest_0502_082337\test\model_epoch050\test_metrics.csv
2025-02-05 09:06:07,527: Stored results at runs\GAGEii-MissingValueTest_0502_082337\test\model_epoch050\test_results.p


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assuming `results` is already loaded with subbasin data.

# Extract subbasin keys
subbasin_keys = list(results.keys())

# Initialize lists to store NSE values and subbasin IDs
nse_values = []
subbasin_ids = []

# Loop through each subbasin and collect NSE values
for key in subbasin_keys:
    
    if 'NSE' not in results[key]['1D']:
        print(f"Skipping {key} as it does not have 1D results.")
        continue
    
    nse = results[key]['1D']['NSE']
    nse_values.append(nse)
    subbasin_ids.append(key)

# Convert to a DataFrame for easier manipulation and visualization
nse_df = pd.DataFrame({'Subbasin': subbasin_ids, 'NSE': nse_values})

# Statistical Summary
summary_stats = nse_df['NSE'].describe()


In [14]:
summary_stats   

count    1.000000
mean     0.745795
std           NaN
min      0.745795
25%      0.745795
50%      0.745795
75%      0.745795
max      0.745795
Name: NSE, dtype: float64

In [18]:
nse_df['Performance'] = nse_df['NSE'].apply(lambda x: 'Excellent' if x > 0.75 else 'Good' if x >= 0.36 else 'Unsatisfactory' if x >= 0 else 'Negative')
nse_df['Performance'].value_counts()

Performance
Good    1
Name: count, dtype: int64

In [7]:

# Plot: Box plot
plt.figure(figsize=(10, 6))
plt.boxplot(nse_df['NSE'], vert=True, patch_artist=True, labels=['NSE'])
plt.title("Distribution of NSE Values Across Subbasins")
plt.ylabel("NSE")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Plot: Histogram
plt.figure(figsize=(10, 6))
plt.hist(nse_df['NSE'], bins=10, edgecolor='black', alpha=0.7)
plt.title("Histogram of NSE Values Across Subbasins")
plt.xlabel("NSE")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Text(0.5, 1.0, 'Test period - NSE 0.746')

In [17]:
# extract observations and simulations
qobs = results['04025500']['1D']['xr']['discharge_obs']
qsim = results['04025500']['1D']['xr']['discharge_sim']

fig, ax = plt.subplots(figsize=(16,10))
ax.plot(qobs['date'], qobs)
ax.plot(qsim['date'], qsim)
ax.set_ylabel("Discharge (m³/s)")
ax.set_title(f"Test period - NSE {results['04025500']['1D']['NSE']:.3f}")